# SQL_WINDOWS

### 1. NTH_VALUE

#### Задача:
Вам доступна база данных, принадлежащая службе такси Uber. Она включает таблицу Rides, которая хранит информацию о поездках пассажиров.



In [ ]:
+--------------+--------+---------------------+
| passenger_id | amount | requested_on        |
+--------------+--------+---------------------+
| 1            | 30     | 2024-01-01 15:45:00 |
| 2            | 25     | 2024-02-12 07:00:00 |
| 3            | 15     | 2024-01-13 10:30:00 |
| 1            | 15     | 2024-02-08 18:15:00 |
| 1            | 35     | 2024-02-05 12:15:00 |
| 4            | 70     | 2024-01-20 11:55:00 |
| 4            | 110    | 2024-01-01 12:40:00 |
| 4            | 25     | 2024-02-01 21:30:00 |
| 3            | 80     | 2024-01-10 23:00:00 |
| 1            | 10     | 2024-01-02 18:10:00 |
+--------------+--------+---------------------+

Первое поле этой таблицы содержит идентификатор пассажира, второе — сумму поездки в долларах, третье — дату и время вызова такси.

**Напишите запрос, который извлекает из предложенной базы данных всю информацию о третьей по счету поездке каждого пассажира.**

#### Решение:

``` sql
WITH ThirdRide AS (
       SELECT 
       NTH_VALUE(passenger_id, 3) OVER wind AS passenger_id,
       NTH_VALUE(amount, 3) OVER wind AS amount,
       NTH_VALUE(requested_on, 3) OVER wind AS requested_on
       FROM Rides
       WINDOW wind AS (PARTITION BY passenger_id ORDER BY requested_on)
)

SELECT DISTINCT passenger_id, amount, requested_on
FROM ThirdRide
WHERE passenger_id IS NOT NULL
```

### 2. LAG

#### Условие:
Вам доступна база данных, принадлежащая банку HSBC. Она включает таблицу Payments, которая хранит информацию о банковских платежах, совершенных клиентами банка.

In [ ]:
+----+---------+---------+--------+---------------------+
| id | user_id | card_id | amount | completed_on        |
+----+---------+---------+--------+---------------------+
| 1  | 1       | 1       | 100    | 2024-01-01 12:00:00 |
| 2  | 1       | 1       | 100    | 2024-01-01 12:06:00 |
| 3  | 2       | 2       | 250    | 2024-01-02 18:00:00 |
| 4  | 3       | 1       | 50     | 2024-01-02 18:05:00 |
| 5  | 2       | 2       | 250    | 2024-01-02 18:08:00 |
| 6  | 3       | 1       | 10     | 2024-01-03 10:00:00 |
| 7  | 3       | 1       | 10     | 2024-01-03 10:10:00 |
| 8  | 1       | 2       | 80     | 2024-01-03 10:00:00 |
| 9  | 1       | 2       | 80     | 2024-01-03 10:10:01 |
| 10 | 4       | 1       | 200    | 2024-01-03 13:00:00 |
+----+---------+---------+--------+---------------------+

Первое поле этой таблицы содержит идентификатор платежа, второе — идентификатор клиента, третье — идентификатор банковской карты клиента, с которой был совершен платеж, четвертое — сумму платежа в долларах, пятое — дату и время совершения платежа.

Иногда в результате различных сбоев банковские платежи случайно совершаются повторно, что приводит к двойному списанию денежных средств с банковской карты. Банк HSBC считает, что платеж по ошибке совершен повторно в том случае, если в течение 10 минут до него был совершен платеж от того же клиента, с той же карты и на ту же сумму. Например, платеж с идентификатором 2 считается совершенным повторно случайно, поскольку 6 минут назад был совершен ровно такой же платеж. Аналогичное справедливо для платежей с идентификаторами 5 и 7.

**Напишите запрос, который извлекает из предложенной базы данных идентификаторы платежей, по ошибке совершенных повторно.**

Поле с идентификатором случайно совершенного повторно платежа должно иметь псевдоним repeat_payment_id.

Записи в результирующей таблице должны быть расположены в порядке возрастания значения поля repeat_payment_id.

#### Решение:

``` sql
WITH cte AS (
    SELECT id, TIMEDIFF(completed_on, (LAG(completed_on) OVER (PARTITION BY user_id, card_id ORDER BY id))) AS timediff,
    amount - LAG(amount) OVER (PARTITION BY user_id, card_id ORDER BY id) AS amountdiff
    FROM Payments 
)

SELECT id AS repeat_payment_id
FROM cte 
WHERE timediff <= TIME('00:10:00') AND amountdiff = 0
ORDER BY repeat_payment_id;
```

### 3. ROW_NUMBER

#### Условие:
Вам доступна база данных, принадлежащая компании Google. Она включает таблицу Measurements, которая хранит результаты определенных измерений, полученных с помощью датчика Google.

In [ ]:
+----+--------+---------------------+
| id | result | received_on         |
+----+--------+---------------------+
| 1  | 1100   | 2024-01-01 12:00:00 |
| 2  | 1000   | 2024-01-01 13:00:00 |
| 3  | 1300   | 2024-01-01 14:00:00 |
| 4  | 1200   | 2024-01-01 15:00:00 |
| 5  | 1100   | 2024-01-01 07:00:00 |
| 6  | 450    | 2024-01-31 08:00:00 |
| 7  | 600    | 2024-01-31 09:00:00 |
| 8  | 650    | 2024-01-31 10:00:00 |
| 9  | 700    | 2024-02-01 07:00:00 |
| 10 | 600    | 2024-02-01 18:00:00 |
+----+--------+---------------------+

Первое поле этой таблицы содержит идентификатор измерения, второе — полученное значение, третье — дату и время измерения.

**Напишите запрос, который разбивает результаты измерений на группы в зависимости от дня, в который они были выполнены, вычисляет в рамках каждой группы сумму результатов нечетных и четных по счету измерений и отображает полученный результат в виде таблицы из трех полей:**

1. measurement_day — день, в который были выполнены измерения
2. odd_measurements_results_sum — сумма результатов нечетных измерений в этот день (результат первого измерения, результат третьего измерения, и так далее)
3. even_measurements_results_sum — сумма результатов четных измерений в этот день (результат второго измерения, результат четвертого измерения, и так далее)

``` sql
WITH EvenResult AS (
    SELECT id, result, received_on, DATE(received_on) AS day,
            ROW_NUMBER() OVER (PARTITION BY DATE(received_on) ORDER BY received_on) num
    FROM Measurements
    ORDER BY received_on
)

SELECT day AS measurement_day, 
       SUM(CASE
               WHEN num % 2 != 0 THEN result
               ELSE 0
           END) AS odd_measurements_results_sum,
       SUM(CASE
               WHEN num % 2 = 0 THEN result
               ELSE 0
            END) AS even_measurements_results_sum
FROM EvenResult
GROUP BY day
```